# Libraries

In [1]:
import requests
import pandas as pd
import time, re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import argparse
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from csv import DictWriter
from threading import Thread
import os, csv, re, queue, time, winsound, traceback
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import argparse
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from csv import DictWriter
from threading import Thread
import os, csv, re, queue, time, winsound, traceback

# Parameter

In [15]:
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--ignore-ssl-errors')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--incognito')
chrome_options.add_argument("--start-maximized")
caps = webdriver.DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "normal"

properties = [
    'supervised',
    'unsupervised',
    'semi-supervised',
    'reinforcement learning',
    'transfer learning',
    'self-supervised learning',
    'multi-task learning',
    'active learning',
    'graph neural networks',
    'convolutional neural networks',
    'recurrent neural networks',
    'physics-informed neural networks (PINNs)',
    'physics-constrained learning',
    'neural operator learning',
    'variational methods',
    'data-driven methods',
    'model-based learning',
    'hybrid methods',
    'stochastic learning',
    'probabilistic methods',
    'meta-learning',
    'optimization-based approaches',
    'adaptive learning',
    'deep learning',
    'machine learning',
    'inverse modeling',
    'domain decomposition methods',
    'multiscale modeling',
    'transferable representations'
]

property_groups = [
    'acoustic wave',
    'seismic wave',
    'electromagnetic wave',
    'elastic wave',
    'ocean wave',
    'shock waves',
    'quantum wave',
    'plasma waves',
    'gravity waves',
    'light waves',
    'surface waves',
    'body waves',
    'fluid dynamics',
    'nonlinear wave',
    'hyperbolic wave equations',
    'wave propagation in anisotropic media',
    'wave scattering',
    'wave dispersion',
    'wave interference',
    'standing waves',
    'traveling waves',
    'wave equations in bounded domains',
    'wave equations in unbounded domains',
    'multi-dimensional wave equations',
    'stochastic wave equations',
    'inverse wave problems',
    'adaptive wave modeling'
]


information_keys = ['title', 'link', 'number_of_citation', 'article_type', 'publisher', 'keyword', 'abstract', 'publish_date', 'publication_title', 'authors', 'affiliations', 'number_of_affiliations', 'countries', 'number_of_countries', 'number_of_found_property_group', 'number_of_found_properties']
FILE_NAME = f'articles_{np.random.randint(10000, 99999)}.csv'

with open(FILE_NAME, 'w', newline='', encoding='utf-8') as file:
    writer = DictWriter(file, fieldnames=information_keys)
    writer.writeheader()

# Scopus

In [16]:
import requests
import pandas as pd
import time

# Define your API key and query
API_KEY = 'cafe5675b6f865a0e9792a6763a7078b'  # Replace with your actual Scopus API key
query = 'TITLE-ABS-KEY ( ( "Physic-informed" OR "PINN" ) AND ( "wave" ) )'

# Scopus API endpoint
url = 'https://api.elsevier.com/content/search/scopus'

# Define headers for authentication
headers = {
    'X-ELS-APIKey': API_KEY,
    'Accept': 'application/json'
}

# Pagination parameters
count = 25  # Adjust based on service level
start = 0
max_results = 1000  # Set an upper limit to prevent too many requests

# Initialize list to hold all articles
all_articles = []

while True:
    # Define the parameters for the current request
    params = {
        'query': query,
        'view': 'STANDARD',  # Retrieves complete metadata
        'count': count,
        'start': start
    }

    # Make the request to Scopus API
    response = requests.get(url, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Extract total number of results on the first request
        if start == 0:
            total_results = int(data.get('search-results', {}).get('opensearch:totalResults', '0'))
            print(f"Total results found: {total_results}")
            if total_results == 0:
                print("No results found for the given query.")
                break

        # Extract information for each article
        entries = data.get('search-results', {}).get('entry', [])
        if not entries:
            print("No more entries to retrieve.")
            break

        for entry in entries:
            # Extract the required fields with safe defaults
            article_title = entry.get('dc:title', 'N/A')
            link_article = entry.get('prism:url', 'N/A')
            number_of_citation = entry.get('citedby-count', 0)
            article_type = entry.get('subtypeDescription', 'N/A')
            publisher = 'Scopus'
            keywords = entry.get('authkeywords', [])
            keywords = ', '.join(keywords) if isinstance(keywords, list) else keywords
            abstract = entry.get('dc:description', 'N/A')
            publication_date = entry.get('prism:coverDate', 'N/A')
            publication_title = entry.get('prism:publicationName', 'N/A')

            # Extract authors and their affiliations
            n_author_informations = len(entry.get('author', []))
            affiliation = entry.get('affiliation', [])
            author = entry.get('author', [])
            count2 = 0
            authors = ''
            countries = ''
            affiliations = ''
            
            for i in range(n_author_informations):
                temp = author[i]['surname'] + ' '  
                if author[i]['given-name'] != None:
                    temp += author[i]['given-name']
                authors += temp

                for af in affiliation:
                    if 'afid' in author[i].keys():
                        if af['afid'] == author[i]['afid'][0]['$']:
                            temp = af['affilname']  + ',' 
                            if af['affiliation-city'] != None:
                                temp += af['affiliation-city'] + ',' 
                            if af['affiliation-country'] != None:
                                temp += af['affiliation-country']
                                countries += af['affiliation-country']
                            affiliations += f'({temp})'

                if count2 < len(author)-1:
                    authors         += ','
                    countries       += ','
                    affiliations    += ','
                count2 += 1

            
            countProperties = 0
            countPropertyGroup = 0
            for keyword in properties:
                countProperties += len(re.findall(rf'\b{re.escape(keyword)}\b', abstract, re.IGNORECASE))
                countProperties += len(re.findall(rf'\b{re.escape(keyword)}\b', article_title, re.IGNORECASE))
            for keyword in property_groups:
                countPropertyGroup += len(re.findall(rf'\b{re.escape(keyword)}\b', article_title, re.IGNORECASE))
                countPropertyGroup += len(re.findall(rf'\b{re.escape(keyword)}\b', abstract, re.IGNORECASE))
                

            # Store the article data in a dictionary
            article = {
                'title': article_title,
                'link': link_article,
                'number_of_citation': number_of_citation,
                'article_type': article_type,
                'publisher': 'Scopus',
                'keyword': keywords,
                'abstract': abstract,
                'publish_date': publication_date,
                'publication_title': publication_title,
                'authors'                       : authors,
                'affiliations'                  : affiliations,
                'number_of_affiliations'        : len(affiliations),
                'countries'                     : countries,
                'number_of_countries'           : len(countries.split(',')),
                'number_of_found_property_group': countPropertyGroup,
                'number_of_found_properties'    : countProperties
            }
            all_articles.append(article)

        print(f"Retrieved {len(entries)} articles. Total collected: {len(all_articles)}.")

        # Update the start parameter for the next batch
        start += count

        # Check if we've retrieved all results or reached the maximum allowed
        if start >= total_results or start >= max_results:
            for col in article.keys():
                print(f"{col}:\n\t\t\t\t{article[col]}")
            print("All available articles have been retrieved.")
            break

        # Optional: Pause between requests to respect rate limits
        time.sleep(1)  # Adjust the sleep duration as needed

    else:
        print(f"Failed to retrieve data: {response.status_code} - {response.text}")
        break

# Convert the collected articles into a pandas DataFrame
df = pd.DataFrame(all_articles)

# Save the DataFrame to an Excel file
output_file = f'scopus_{len(all_articles)}_articles.xlsx'
df.to_excel(output_file, index=False)

print(f"Total articles retrieved: {len(all_articles)}. Data saved to '{output_file}'.")


Total results found: 361
Retrieved 25 articles. Total collected: 25.
Retrieved 25 articles. Total collected: 50.
Retrieved 25 articles. Total collected: 75.
Retrieved 25 articles. Total collected: 100.
Retrieved 25 articles. Total collected: 125.
Retrieved 25 articles. Total collected: 150.
Retrieved 25 articles. Total collected: 175.
Retrieved 25 articles. Total collected: 200.
Retrieved 25 articles. Total collected: 225.
Retrieved 25 articles. Total collected: 250.
Retrieved 25 articles. Total collected: 275.
Retrieved 25 articles. Total collected: 300.
Retrieved 25 articles. Total collected: 325.
Retrieved 25 articles. Total collected: 350.
Retrieved 11 articles. Total collected: 361.
title:
				Physics-based machine learning inversion of subsurface elastic properties
link:
				https://api.elsevier.com/content/abstract/scopus_id/85084019415
number_of_citation:
				2
article_type:
				Conference Paper
publisher:
				Scopus
keyword:
				
abstract:
				N/A
publish_date:
				2019-06-03


# Sciencedirect

In [18]:
import requests
import pandas as pd
import time

# Define your API key and query
API_KEY = '2dcf4608be9c0c1f712754390c03c123'  # Replace with your actual Scopus API key
query2 = '("physic-informed" ) AND ( "wave" ) '
# Scopus API endpoint
url = 'https://api.elsevier.com/content/search/sciencedirect'

# Define headers for authentication
headers = {
    'X-ELS-APIKey': API_KEY,
    'Accept': 'application/json'
}

# Pagination parameters
count = 25  # Adjust based on service level
start = 0
max_results = 1000  # Set an upper limit to prevent too many requests

# Initialize list to hold all articles
all_articles = []
link_articles = []
while True:
    # Define the parameters for the current request
    params = {
        'query': query2,
        'apiKey': API_KEY,
        'httpAccept': 'application/json',
        'view': 'COMPLETE'  # This ensures more detailed information, including abstracts, is retrieved
    }

    # Make the request to Scopus API
    response = requests.get(url, headers=headers, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Extract total number of results on the first request
        if start == 0:
            total_results = int(data.get('search-results', {}).get('opensearch:totalResults', '0'))
            print(f"Total results found: {total_results}")
            if total_results == 0:
                print("No results found for the given query.")
                break

        # Extract information for each article
        entries = data.get('search-results', {}).get('entry', [])
        if not entries:
            print("No more entries to retrieve.")
            break

        
        for entry in entries:
            link_article = 'https://doi.org/' + entry.get('prism:doi')
            link_articles.append(link_article)

        # Update the start parameter for the next batch
        start += count

        # Check if we've retrieved all results or reached the maximum allowed
        if start >= total_results or start >= max_results:
            # for col in article.keys():
            #     print(f"{col}:\n\t\t\t\t{article[col]}")
            print("All available articles have been retrieved.")
            break

        # Optional: Pause between requests to respect rate limits
        time.sleep(1)  # Adjust the sleep duration as needed

    else:
        print(f"Failed to retrieve data: {response.status_code} - {response.text}")
        break



Failed to retrieve data: 401 - {"service-error":{"status":{"statusCode":"AUTHORIZATION_ERROR","statusText":"The requestor is not authorized to access the requested view or fields of the resource"}}}


# Manual Sciencedirect

In [19]:
FILE_NAME = f'scopus_361_articles.csv'

with open(FILE_NAME, 'w', newline='', encoding='utf-8') as file:
    writer = DictWriter(file, fieldnames=information_keys)
    writer.writeheader()

count_article = 0
trial = 0
failed_retrieve = 0

service = Service(ChromeDriverManager().install())
for link_article in link_articles:
    driver = webdriver.Chrome(service=service, options=chrome_options)
    try:
        trial += 1
        driver.get(link_article)
        time.sleep(5)
        show_more_button =  WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH, '//*[@id="show-more-btn"]'
                )
            )
        )
        show_more_button.click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        article_title   = soup.find('span', class_='title-text').get_text()
        abstract        = soup.find('div', class_='abstract author').get_text().replace('Abstract', '')
        publication_title = soup.find('meta', attrs={'name': 'citation_journal_title'})['content']
        cited_info = driver.find_element(By.ID, 'citing-articles-header')
        driver.execute_script("arguments[0].scrollIntoView(true);", cited_info)
        number_of_citation   = soup.find('div', class_='CitedBy')
        driver.close()

        if number_of_citation != None:
            number_of_citation = number_of_citation.get_text()
            number_of_citation   = re.search(r'\((\d+)\)', number_of_citation).group(1)
        else:
            number_of_citation = soup.find(id="citing-articles-header")
            if number_of_citation != None:
                number_of_citation = number_of_citation.get_text()
                number_of_citation   = re.search(r'\((\d+)\)', number_of_citation).group(1)
            else:
                number_of_citation = 'none'


        article_type = 'none'
        keywords = soup.find('div', class_='keywords-section').find_all('div', class_='keyword')
        keywords = ','.join(keyword.get_text() for keyword in keywords)


        author_informations = soup.find('div', class_='wrapper')
        if author_informations == None:
            author_informations = soup.find('div', class_='wrapper truncated')

        publication_date = author_informations.find('p').get_text()
        publication_date = publication_date.split(',')
        publication_date = publication_date[len(publication_date)-1].replace(' Version of Record ', '')
        publication_date = publication_date.replace('.', '')

        author_informations = author_informations.find('div', id='author-group')
        author_informations = author_informations.find_all('button', class_='button-link button-link-secondary button-link-underline')

        affiliation_informations = soup.find_all('dl', class_='affiliation')
        temp = {}
        
        for affiliation_information in affiliation_informations:
            if affiliation_information.find('sup') != None:
                temp[affiliation_information.find('sup').get_text()] = affiliation_information.find('dd').get_text()
            else:
                temp = affiliation_information.find('dd').get_text()


        authors = ''
        countries = ''
        affiliations = ''
        count = 0

        for author_information in author_informations:
            author = author_information.find('span', 'text surname').get_text() + ' '
            author += author_information.find('span', 'given-name').get_text()
            authors += author

            author_affiliation = author_information.find('sup')
            if author_affiliation != None:
                author_affiliation = author_affiliation.get_text()
                affiliations += f'({temp[author_affiliation]})'
                country = temp[author_affiliation].split(',')[len(temp[author_affiliation].split(','))-1]
            else:
                affiliations += f'({temp})'
                country += temp.split(',')[len(temp.split(','))-1]

            countries += country

            if count < len(author_informations)-1:
                authors         += ','
                countries       += ','
                affiliations    += ','

            count += 1

        countProperties = 0
        countPropertyGroup = 0
        for keyword in properties:
            countProperties += len(re.findall(rf'\b{re.escape(keyword)}\b', abstract, re.IGNORECASE))
            countProperties += len(re.findall(rf'\b{re.escape(keyword)}\b', article_title, re.IGNORECASE))
        for keyword in property_groups:
            countPropertyGroup += len(re.findall(rf'\b{re.escape(keyword)}\b', article_title, re.IGNORECASE))
            countPropertyGroup += len(re.findall(rf'\b{re.escape(keyword)}\b', abstract, re.IGNORECASE))

        row_data = {
            'title'                         : article_title,
            'link'                          : link_article,
            'number_of_citation'            : number_of_citation,
            'article_type'                  : article_type,
            'publisher'                     : 'Sciencedirect',
            'keyword'                       : keywords,
            'abstract'                      : abstract,
            'publish_date'                  : publication_date,
            'publication_title'             : publication_title,
            'authors'                       : authors,
            'affiliations'                  : affiliations,
            'number_of_affiliations'        : len(affiliations),
            'countries'                     : countries,
            'number_of_countries'           : len(countries.split(',')),
            'number_of_found_property_group': countPropertyGroup,
            'number_of_found_properties'    : countProperties
        }

        
        with open(FILE_NAME, 'a', newline='', encoding='utf-8') as file:
            writer = DictWriter(file, fieldnames=information_keys)
            writer.writerow(row_data)

        count_article += 1
        print("\n"*3)
        print("=="*100)
        print(f"Articles successfully ({((count_article/233)*100):.2f}%) | Successfully retrieve : ({count_article}) | Failed : ({failed_retrieve}) | Total trial : {trial}")
        print("\n"*1)
        
    except Exception as e:
        failed_retrieve += 1
        print("== ! Error ! =="*10)
        print("\n"*1)
        print(f"Failed to retrieve article: \n{article_title}\n{link_article}\n{e}")
        traceback.print_exc()
        print("\n"*1)
        print("== ! Error ! =="*10)
        print("\n"*1)

In [ ]:
link_article = 'https://doi.org/10.1016/j.future.2023.08.006'
driver = webdriver.Chrome(service=service, options=chrome_options)

trial += 1
driver.get(link_article)
show_more_button =  WebDriverWait(driver, 30).until(
    EC.element_to_be_clickable(
        (
            By.XPATH, '//*[@id="show-more-btn"]'
        )
    )
)
show_more_button.click()
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

article_title   = soup.find('span', class_='title-text').get_text()
abstract        = soup.find('div', class_='abstract author').get_text().replace('Abstract', '')
publication_title = soup.find('meta', attrs={'name': 'citation_journal_title'})['content']
cited_info = driver.find_element(By.ID, 'citing-articles-header')
driver.execute_script("arguments[0].scrollIntoView(true);", cited_info)
number_of_citation   = soup.find('div', class_='CitedBy')

if number_of_citation != None:
    number_of_citation = number_of_citation.get_text()
    number_of_citation   = re.search(r'\((\d+)\)', number_of_citation).group(1)
else:
    number_of_citation = soup.find(id="citing-articles-header")
    if number_of_citation != None:
        number_of_citation = number_of_citation.get_text()
        number_of_citation   = re.search(r'\((\d+)\)', number_of_citation).group(1)
    else:
        number_of_citation = 'none'


article_type = 'none'
keywords = soup.find('div', class_='keywords-section').find_all('div', class_='keyword')
keywords = ','.join(keyword.get_text() for keyword in keywords)


author_informations = soup.find('div', class_='wrapper')
if author_informations == None:
    author_informations = soup.find('div', class_='wrapper truncated')

publication_date = author_informations.find('p').get_text()
publication_date = publication_date.split(',')
publication_date = publication_date[len(publication_date)-1].replace(' Version of Record ', '')
publication_date = publication_date.replace('.', '')

author_informations = author_informations.find('div', id='author-group')
author_informations = author_informations.find_all('button', class_='button-link button-link-secondary button-link-underline')

affiliation_informations = soup.find_all('dl', class_='affiliation')
temp = {}

for affiliation_information in affiliation_informations:
    if affiliation_information.find('sup') != None:
        temp[affiliation_information.find('sup').get_text()] = affiliation_information.find('dd').get_text()
    else:
        temp = affiliation_information.find('dd').get_text()


authors = ''
countries = ''
affiliations = ''
count = 0

for author_information in author_informations:
    author = author_information.find('span', 'text surname').get_text() + ' '
    author += author_information.find('span', 'given-name').get_text()
    authors += author

    author_affiliation = author_information.find('sup')
    if author_affiliation != None:
        author_affiliation = author_affiliation.get_text()
        affiliations += f'({temp[author_affiliation]})'
        country = temp[author_affiliation].split(',')[len(temp[author_affiliation].split(','))-1]
    else:
        affiliations += f'({temp})'
        country += temp.split(',')[len(temp.split(','))-1]

    countries += country

    if count < len(author_informations)-1:
        authors         += ','
        countries       += ','
        affiliations    += ','

    count += 1

countProperties = 0
countPropertyGroup = 0
for keyword in properties:
    countProperties += len(re.findall(rf'\b{re.escape(keyword)}\b', abstract, re.IGNORECASE))
    countProperties += len(re.findall(rf'\b{re.escape(keyword)}\b', article_title, re.IGNORECASE))
for keyword in property_groups:
    countPropertyGroup += len(re.findall(rf'\b{re.escape(keyword)}\b', article_title, re.IGNORECASE))
    countPropertyGroup += len(re.findall(rf'\b{re.escape(keyword)}\b', abstract, re.IGNORECASE))

row_data = {
    'title'                         : article_title,
    'link'                          : link_article,
    'number_of_citation'            : number_of_citation,
    'article_type'                  : article_type,
    'publisher'                     : 'Sciencedirect',
    'keyword'                       : keywords,
    'abstract'                      : abstract,
    'publish_date'                  : publication_date,
    'publication_title'             : publication_title,
    'authors'                       : authors,
    'affiliations'                  : affiliations,
    'number_of_affiliations'        : len(affiliations),
    'countries'                     : countries,
    'number_of_countries'           : len(countries.split(',')),
    'number_of_found_property_group': countPropertyGroup,
    'number_of_found_properties'    : countProperties
}

In [ ]:
author_informations = soup.find('div', class_='wrapper')
author_informations

In [ ]:
author_informations = soup.find('div', class_='wrapper truncated')
author_informations

In [ ]:
print(soup.prettify())

<html data-sd-ui-what-input="mouse" lang="en-US">
 <head>
  <meta content="S2096579622001139" name="citation_pii"/>
  <meta content="2096-5796" name="citation_issn"/>
  <meta content="6" name="citation_volume"/>
  <meta content="16" name="citation_lastpage"/>
  <meta content="1" name="citation_issue"/>
  <meta content="Elsevier" name="citation_publisher"/>
  <meta content="1" name="citation_firstpage"/>
  <meta content="" name="citation_fulltext_world_readable"/>
  <meta content="Virtual Reality &amp; Intelligent Hardware" name="citation_journal_title"/>
  <meta content="JOUR" name="citation_type"/>
  <meta content="10.1016/j.vrih.2022.10.002" name="citation_doi"/>
  <meta content="10.1016/j.vrih.2022.10.002" name="dc.identifier"/>
  <meta content="Full-length article" name="citation_article_type"/>
  <meta content="Wireless sensor networks (WSN) gather information and sense information samples in a certain region and communicate these readings to a base station (…" property="og:descri

In [ ]:

soup.find(id="citing-articles-header").get_text()

In [ ]:
article_title

In [ ]:
entry

{'@_fa': 'true',
 'load-date': '2022-11-02T00:00:00.000Z',
 'link': [{'@_fa': 'true',
   '@ref': 'self',
   '@href': 'https://api.elsevier.com/content/article/pii/S0167739X22003272'},
  {'@_fa': 'true',
   '@ref': 'scidir',
   '@href': 'https://www.sciencedirect.com/science/article/pii/S0167739X22003272?dgcid=api_sd_search-api-endpoint'}],
 'dc:identifier': 'DOI:10.1016/j.future.2022.10.011',
 'prism:url': 'https://api.elsevier.com/content/article/pii/S0167739X22003272',
 'dc:title': 'DAG block: Trust aware load balanced routing and lightweight authentication encryption in WSN',
 'dc:creator': 'Revanesh M.',
 'prism:publicationName': 'Future Generation Computer Systems',
 'prism:volume': '140',
 'prism:coverDate': '2023-03-31',
 'prism:startingPage': '402',
 'prism:endingPage': '421',
 'prism:doi': '10.1016/j.future.2022.10.011',
 'openaccess': False,
 'pii': 'S0167739X22003272',
 'authors': {'author': [{'$': 'Revanesh M.'},
   {'$': 'John M. Acken'},
   {'$': 'V. Sridhar'}]}}